#### Flow monitor

In [4]:
import xml.etree.ElementTree as ET
import pandas as pd

# Load the XML file
flowmon_path="/home/nourhen_dev/repos/ns-3-allinone/ns-3.41/flowmon_three_nodes.xml"
tree = ET.parse(flowmon_path)  # Replace with your file path
root = tree.getroot()

# Prepare a list to collect parsed flow data
flow_data = []

# Iterate over all <Flow> elements inside <FlowStats>
#---------------nano seconds to second & bytes to Megabytes
for flow in root.find('FlowStats').findall('Flow'):
    flow_id = int(flow.attrib['flowId'])
    time_first_tx = float(flow.attrib['timeFirstTxPacket'].replace('ns', '').replace('+', ''))*1E-9
    time_first_rx = float(flow.attrib['timeFirstRxPacket'].replace('ns', '').replace('+', ''))*1E-9
    time_last_tx = float(flow.attrib['timeLastTxPacket'].replace('ns', '').replace('+', ''))*1E-9
    time_last_rx = float(flow.attrib['timeLastRxPacket'].replace('ns', '').replace('+', ''))*1E-9
    delay_sum = float(flow.attrib['delaySum'].replace('ns', '').replace('+', ''))*1E-9
    jitter_sum = float(flow.attrib['jitterSum'].replace('ns', '').replace('+', ''))*1E-9
    last_delay = float(flow.attrib['lastDelay'].replace('ns', '').replace('+', ''))*1E-9
    tx_bytes = int(flow.attrib['txBytes'])
    rx_bytes = int(flow.attrib['rxBytes'])
    tx_packets = int(flow.attrib['txPackets'])
    rx_packets = int(flow.attrib['rxPackets'])
    lost_packets = int(flow.attrib['lostPackets'])
    times_forwarded = int(flow.attrib['timesForwarded'])
    duration = time_last_rx - time_first_tx
    throughput_Mbps = (int(flow.attrib['rxBytes']) * 8) / (duration * 1e6)  # bits/sec → Mbps


    flow_data.append({
        'flow_id': flow_id,
        'time_first_tx_s': time_first_tx,
        'time_first_rx_s': time_first_rx,
        'time_last_tx_s': time_last_tx,
        'time_last_rx_s': time_last_rx,
        'delay_sum_s': delay_sum,
        'jitter_sum_s': jitter_sum,
        'last_delay_s': last_delay,
        'tx_bytes': tx_bytes,
        'rx_bytes': rx_bytes,
        'tx_packets_Mb': tx_packets,
        'rx_packets_Mb': rx_packets,
        'lost_packets_Mb': lost_packets,
        'times_forwarded': times_forwarded,
        'duration':duration,
        'throughput_Mbps':throughput_Mbps
    })

classifier_info = {}
for flow in root.find('Ipv4FlowClassifier').findall('Flow'):
    flow_id = int(flow.attrib['flowId'])
    classifier_info[flow_id] = {
        'src': flow.attrib['sourceAddress'],
        'dst': flow.attrib['destinationAddress'],
        'src_port': flow.attrib['sourcePort'],
        'dst_port': flow.attrib['destinationPort'],
        'protocol': flow.attrib['protocol']
    }

# Merge with previous stats if needed
for row in flow_data:
    fid = row['flow_id']
    if fid in classifier_info:
        row.update(classifier_info[fid])
df_flowmon = pd.DataFrame(flow_data)
df_flowmon

,flow_id,time_first_tx_s,time_first_rx_s,time_last_tx_s,time_last_rx_s,delay_sum_s,jitter_sum_s,last_delay_s,tx_bytes,rx_bytes,...,rx_packets_Mb,lost_packets_Mb,times_forwarded,duration,throughput_Mbps,src,dst,src_port,dst_port,protocol
0,1,1.18980,1.19188,19.9997,20.0018,33.9828,0.0,0.002084,17151808,17151808,...,16304,0,0,18.81200,7.293986,10.1.3.1,10.1.3.2,49153,10,17
1,2,1.38681,1.38889,19.7109,19.7129,24.8868,0.0,0.002084,12560880,12560880,...,11940,0,0,18.32609,5.483278,10.1.1.1,10.1.1.2,49153,9,17
